In [1]:
import csv
import pandas as pd
import numpy as np
from collections import defaultdict

In [17]:
df_Dino = pd.read_csv("Input Data/detections_CME.csv", header = None)
M_Dino = df_Dino.to_numpy()
M_Dino[:, 4] = M_Dino[:, 4]-1

df_CME = pd.read_csv("Input Data/detections_Dino.csv")
df_CME = df_CME[["timepoint","label_id","x","y","z","intensity","max_intensity"]]
df_CME = df_CME.drop("max_intensity", axis=1)
M_CME = df_CME.to_numpy()
M_CME[:, 0] += 1

display(df_CME)

comp_list = np.zeros((len(M_CME),15), dtype=object)

,timepoint,label_id,x,y,z,intensity
0,0,0,442.55520,352.933350,96.218920,170.4
1,0,1,261.47263,63.179287,2.617795,344.5
2,0,2,255.29057,122.914760,2.239835,298.0
3,0,3,257.73523,180.409400,2.184479,436.8
4,0,4,259.04593,194.088610,2.714999,420.8
...,...,...,...,...,...,...
178317,98,1101,519.40436,664.726260,165.479970,294.8
178318,98,1102,576.44190,678.389650,166.223020,243.0
178319,98,1103,571.13990,687.164900,164.807560,300.0
178320,98,1104,517.80750,694.140750,164.642750,260.0


In [3]:
t_vec = defaultdict(list)
threshold = 5
multi_match_list = []
for vec_Dino in M_Dino:
    t_vec[vec_Dino[0]].append(vec_Dino)

for t_val in t_vec:
    t_vec[t_val] = np.array(t_vec[t_val])

for i, vec_CME in enumerate(M_CME):
    t_val = vec_CME[0]
    dino_group = t_vec[t_val]
    diffs = dino_group[:, 2:5] - vec_CME[2:5]
    dists = np.linalg.norm(diffs, axis=1)
    min_idx = np.argmin(dists)
    min_dist = dists[min_idx]
    best_vec_Dino = dino_group[min_idx]
    comp_list[i, 0:6] = vec_CME
    comp_list[i, 6:12] = best_vec_Dino
    comp_list[i, 12] = min_dist

    # New list by threshold
    below = np.where(dists < threshold)[0]
    if len(below) > 0:
        dino_ids = ",".join(str(int(x)) for x in dino_group[below, 1])
        dists_str = ",".join(f"{dists[j]:.2f}" for j in below)
    else:
        dino_ids = ""
        dists_str = ""
    comp_list[i, 13] = dino_ids
    comp_list[i, 14] = dists_str

multi_match_array = np.array(multi_match_list, dtype=object)

In [12]:
df_comp = pd.DataFrame(comp_list, columns = ["t","ID (Dino)", "x (Dino)", "y (Dino)", 
                                             "z (Dino)", "FI (Dino)", "t_ig", "ID (CME)", 
                                             "x (CME)", "y (CME)", "z (CME)", 
                                             "FI (CME)", "Distance", "Multi ID (CME)", "Multi Distance (CME)"])

#df_comp = df_comp.drop("t_ig", axis=1)
df_sorted = df_comp.sort_values(by=["ID (Dino)"], ascending=[True])

In [5]:
df_sorted.to_csv("Output Data/DetectionComparison.csv", index=False)

In [13]:

#display(df_comp[df_comp["Distance"]<3.5])
display(df_sorted)

,t,ID (Dino),x (Dino),y (Dino),z (Dino),FI (Dino),t_ig,ID (CME),x (CME),y (CME),z (CME),FI (CME),Distance,Multi ID (CME),Multi Distance (CME)
0,1.0,0.0,442.5552,352.93335,96.21892,170.4,1.0,584.0,458.53,353.38,89.863,75.586,17.198589,,
43284,26.0,0.0,518.6719,375.70316,114.02337,247.8,26.0,868.0,496.26,367.14,143.96,58.899,38.364343,,
49429,30.0,0.0,524.1598,380.44925,117.25394,247.8,30.0,919.0,506.38,386.67,153.43,132.8,40.78635,,
16603,11.0,0.0,503.32837,363.5962,105.080536,258.2,11.0,614.0,462.06,361.66,94.63,121.18,42.615032,,
85354,51.0,0.0,415.91818,377.864,106.96409,131.0,51.0,637.0,420.17,378.57,102.95,54.681,5.889765,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134307,71.0,2909.0,549.0457,682.47906,166.99991,284.0,71.0,978.0,556.19,686.22,161.65,224.61,9.677665,,
134308,71.0,2910.0,525.6004,688.05096,166.6149,268.8,71.0,917.0,537.66,696.65,154.37,109.59,19.217571,,
134309,71.0,2911.0,521.4097,693.00433,165.48357,259.0,71.0,917.0,537.66,696.65,154.37,109.59,20.021853,,
134310,71.0,2912.0,562.9968,695.99603,166.99905,279.2,71.0,978.0,556.19,686.22,161.65,224.61,13.058163,,
